(masses_bfr_ntbk)=

# Masses and binary fraction

This tutorial shows how we can use methods available in the [Synthetic](asteca.Synthetic) class to estimate for a given observed cluster:

- The individual stellar masses and their probability of being binary systems,
- the total binary fraction of the cluster, and 
- the different masses associated to the cluster

We start by loading the cluster file and a set of isochrones, and instantiating a synthetic object which we calibrate with the cluster's data:

In [5]:
import asteca
import pandas as pd
import numpy as np

obs_df = pd.read_csv("../_static/cluster.csv")

# Load cluster file
my_cluster = asteca.Cluster(
    ra=obs_df["RA_ICRS"],
    dec=obs_df["DE_ICRS"], 
    magnitude=obs_df["Gmag"],
    e_mag=obs_df["e_Gmag"],
    color=obs_df["BP-RP"],
    e_color=obs_df["e_BP-RP"],
    verbose=2
)

# Load isochrones
isochs = asteca.Isochrones(
    model='parsec',
    isochs_path="../_static/parsec/",
    magnitude="Gmag",
    color=("G_BPmag", "G_RPmag"),
    magnitude_effl=6390.7,
    color_effl=(5182.58, 7825.08),
    verbose=2
)

# Create Synthetic cluster object
synthcl = asteca.Synthetic(isochs, seed=457304, verbose=2)

# Calibrate the `synthcl` object
synthcl.calibrate(my_cluster)


Instantiating cluster...
Columns read   : RA, DEC, Magnitude, e_mag, Color, e_color
N_stars        : 2759
N_clust_min    : 25
N_clust_max    : 5000
Cluster object generated

Instantiating isochrones...
Model          : PARSEC
N_files        : 1
N_mets         : 3
N_ages         : 11
N_isochs       : 2000
z    range     : [0.01, 0.02]
loga range     : [7.0, 9.5]
Magnitude      : Gmag
Color          : G_BPmag-G_RPmag
Isochrone object generated

Instantiating synthetic...
Default params : {'met': 0.0152, 'loga': 8.0, 'alpha': 0.09, 'beta': 0.94, 'Rv': 3.1, 'DR': 0.0, 'Av': 0.2, 'dm': 9.0}
Extinction law : CCMO
Diff reddening : uniform
IMF            : chabrier_2014
Max init mass  : 10000
Gamma dist     : D&K
Random seed    : 457304
Synthetic clusters object generated
N_stars_obs    : 2759
Max magnitude  : 19.00
Error distribution loaded


For all three analyses mentioned above, the first step is to call the [get_models()](asteca.Synthetic.get_models) method. This method requires two arguments: `model` which is a dictionary of parameters to be fitted and a `model_std` dictionary which contains the uncertainties (standard deviations) associated to each parameter in the `model` dictionary. For example:

In [12]:
# Values for the fundamental parameters associated to the observed cluster
model = {"met": 0.015, "loga": 8., "dm": 8., "Av": 0.05}

# Uncertainties for each fundamental parameter
model_std = {"met": 0.001, "loga": 0.2, "dm": 0.2, "Av": 0.03}

# Call the method
synthcl.get_models(model, model_std)


Generate synthetic models...
N_models       : 200
Attributes stored in Synthetic object


```{note}
Notice that some of the parameters in `model_std` have STDDEV values of `0.0`. This means that these parameters are fixed to the values given in the `model` dictionary.
```

This method will store in the `synthcl` object a number of synthetic clusters, sampled from a normal distribution centered on `model` values with STDDEVs taken from the `model_std` values. By default `200` models are generated, which can be changed via the `N_models` argument of the method.

After calling this method, the individual stellar masses and binarity, cluster total mass and cluster total binarity fraction can be estimated as shown in the following sub-sections.

## Stellar masses and binarity

An estimation of the observed stars individual masses and their probability of belonging to a binary system can be achieved via the [stellar_masses()](asteca.Synthetic.stellar_masses) method:

In [13]:
df_masses_bprob = synthcl.stellar_masses()

# Convert to pandas DataFrame for prettier printing
print(pd.DataFrame(df_masses_bprob))


Stellar masses and binary probabilities estimated
            m1    m1_std        m2    m2_std  binar_prob
0     0.445917  0.051379  0.250017  0.107283       0.370
1     1.231061  0.167481  0.736810  0.288590       0.860
2     0.533227  0.050551  0.310715  0.093626       0.280
3     0.933417  0.116045  0.425366  0.148135       0.465
4     0.700783  0.061533  0.342416  0.127329       0.130
...        ...       ...       ...       ...         ...
2754  0.267549  0.034917  0.187424  0.062576       0.740
2755       NaN       NaN       NaN       NaN         NaN
2756  0.583159  0.053920  0.248445  0.064690       0.045
2757  0.516960  0.053511  0.212998  0.000000       0.005
2758  0.293646  0.050600       NaN       NaN       0.000

[2759 rows x 5 columns]


/home/gabriel/Github/ASteCA/ASteCA/asteca/asteca/synthetic.py:559: UserWarning: 
N=36 stars found with no valid photometric data. These will be assigned 'nan' values
for masses and binarity probability
  warnings.warn(


The returned variable `df_masses_bprob` is a dictionary containing the columns `m1, m1_std, m2, m2_std, binar_prob`. These columns represent, for each observed star in the cluster under analysis, estimates for:

- `m1`: primary mass
- `m1_std`: uncertainty of the primary mass
- `m2`: secondary mass (under the assumption that this star belongs to a binary
  system)
- `m2_std`: uncertainty of the secondary mass
- `binar_prob`: probability of being a binary system 

If an observed star has `binar_prob=0`, i.e. a zero probability of being a binary system, then the mass value for its secondary star `m2` is a `NaN` value as no secondary star could be assigned to it. If any observed star contains invalid photometric data,  they will be assigned `NaN` values for masses and binarity probability.

## Total binary fraction

Since the fraction of synthetic binary systems is handled through the `alpha, beta` parameters, there is no *binary fraction* parameter than can be fitted using the synthetic clusters. This parameter needs to be estimated separately via the [binary_fraction()](asteca.Synthetic.binary_fraction) method as follows:

In [15]:
# Obtain distribution of the total binary fraction
bfr_med, bfr_std = synthcl.binary_fraction(df_masses_bprob['binar_prob'])

Binary fraction: 0.446 +/- 0.007


The only argument required for this method is the distribution of binary probabilities `df_masses_bprob['binar_prob']` obtained in the previous section via the [stellar_masses()](asteca.Synthetic.stellar_masses) method.

The `bfr_med, bfr_std`  variables will store the median and STDDEV values for the total binary fraction estimate for the cluster.

## Cluster masses

As explained in the [Mass and binarity](synthetic_mod) section, **ASteCA** estimates several different masses associated to an observed cluster. These are:

- {math}`M_{init}`: total initial mass (at the moment of cluster's birth)
- {math}`M_{actual}`: actual mass of the cluster (even the low mass portion we do not observe)
- {math}`M_{obs}`: observed mass (sum of individual stellar masses)
- {math}`M_{phot}`: mass **unobserved** due to photometric effects (i.e: the low mass stars beyond the maximum magnitude cut)
- {math}`M_{evol}`: mass **lost** via stellar evolution
- {math}`M_{dyn}`: mass **lost** through dynamical effects (or *dissolution*)

This process is performed via the [cluster_masses()](asteca.Synthetic.cluster_masses) method:

In [16]:
masses_dict = synthcl.cluster_masses()


Mass values estimated


The returned dictionary contains arrays with the distributions of masses for each mass described above. Their median and STDDEV values can be extracted for example with:

In [17]:
# Print the median mass values and their STDDEVs
for k, arr in masses_dict.items():
    print("{:<8}: {:.0f}+/-{:.0f}".format(k, np.median(arr), np.std(arr)))

M_init  : 4312+/-696
M_actual: 3167+/-327
M_obs   : 2795+/-180
M_phot  : 380+/-160
M_evol  : 867+/-240
M_dyn   : 307+/-188
